In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the dataset
data = pd.read_csv('../data/wnba_pbp_2018.csv') 

# Display basic information about the dataset
print(data.info())

# Display the first few rows of the dataset
print(data.head())

# Describe the dataset to get statistical summaries of numerical columns
print(data.describe())

# Check for missing values
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84811 entries, 0 to 84810
Data columns (total 61 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       84811 non-null  int64  
 1   id                               84811 non-null  int64  
 2   sequence_number                  84811 non-null  int64  
 3   type_id                          84811 non-null  int64  
 4   type_text                        84811 non-null  object 
 5   text                             84808 non-null  object 
 6   away_score                       84811 non-null  int64  
 7   home_score                       84811 non-null  int64  
 8   period_number                    84811 non-null  int64  
 9   period_display_value             84811 non-null  object 
 10  clock_display_value              84811 non-null  object 
 11  scoring_play                     84811 non-null  bool   
 12  score_value       

In [ ]:
# Generate histograms for all numerical columns to understand distributions
data.hist(figsize=(12, 10))
plt.show()

In [ ]:
# Check data types of each column
print(data.dtypes)

In [ ]:
# Select only the numeric columns of the DataFrame
numeric_data = data.select_dtypes(include=[np.number])

# Now compute the correlation matrix
correlation_matrix = numeric_data.corr()

In [ ]:
# Generate the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

In [3]:
print(data.head(10))

   Unnamed: 0           id  sequence_number  type_id             type_text  \
0           1   4010796954                4      615              Jumpball   
1           2   4010796957                7    20132   Step Back Jump Shot   
2           3   4010796959                9       12           Kicked Ball   
3           4  40107969510               10    30092             Jump Shot   
4           5  40107969511               11      156     Offensive Rebound   
5           6  40107969512               12       70   Shot Clock Turnover   
6           7  40107969513               13    20114  Turnaround Jump Shot   
7           8  40107969514               14      156     Offensive Rebound   
8           9  40107969515               15    20094              Tip Shot   
9          10  40107969516               16      156     Offensive Rebound   

                                                text  away_score  home_score  \
0  Breanna Stewart vs. LaToya Sanders (Elena Dell...         

In [7]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.11/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sentence_transformers import SentenceTransformer
import faiss

In [12]:
# Get column names
column_names = data.columns.tolist()
print(column_names)

['Unnamed: 0', 'id', 'sequence_number', 'type_id', 'type_text', 'text', 'away_score', 'home_score', 'period_number', 'period_display_value', 'clock_display_value', 'scoring_play', 'score_value', 'team_id', 'athlete_id_1', 'athlete_id_2', 'athlete_id_3', 'wallclock', 'shooting_play', 'coordinate_x_raw', 'coordinate_y_raw', 'season', 'season_type', 'away_team_id', 'away_team_name', 'away_team_mascot', 'away_team_abbrev', 'away_team_name_alt', 'home_team_id', 'home_team_name', 'home_team_mascot', 'home_team_abbrev', 'home_team_name_alt', 'home_team_spread', 'game_spread', 'home_favorite', 'game_spread_available', 'game_id', 'qtr', 'time', 'clock_minutes', 'clock_seconds', 'half', 'game_half', 'lead_qtr', 'lead_game_half', 'start_quarter_seconds_remaining', 'start_half_seconds_remaining', 'start_game_seconds_remaining', 'game_play_number', 'end_quarter_seconds_remaining', 'end_half_seconds_remaining', 'end_game_seconds_remaining', 'period', 'lag_qtr', 'lag_game_half', 'coordinate_x', 'coor

In [14]:
data['combined_text'] = data.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [16]:
texts = data['combined_text'].tolist()

In [17]:
from sentence_transformers import SentenceTransformer

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert text data to vectors (embeddings)
text_embeddings = model.encode(texts)

/opt/homebrew/Cellar/jupyterlab/4.0.11/libexec/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
import faiss

# Number of dimensions of the embeddings
d = text_embeddings.shape[1]

# Build the index
index = faiss.IndexFlatL2(d)  # L2 distance (Euclidean distance)

# Add vectors to the index
index.add(text_embeddings)

# Save the index to a file
faiss.write_index(index, '../data/vector_store.index')

In [44]:
# Load the index from the file
index = faiss.read_index('../data/vector_store.index')

# Convert your query into a vector
#query = "Who made the first scoring play in the first quarter?"
#query = "What was the home team’s score at the end of the first quarter?"
query = "Which player made an 18-foot step back jumpshot?"
#query = "Describe the events that happened in the first five minutes of the game."
#query = "List all the jump shots made in the game."
query_embedding = model.encode([query])

# Search the vector store
k = 5  # Number of nearest neighbors
D, I = index.search(query_embedding, k)

# I contains the indices of the nearest neighbors
print("Indices of the nearest neighbors:", I)

# Retrieve the rows corresponding to the indices
result_rows = data.iloc[I[0]].to_dict(orient='records')
print("Query results:", result_rows)

Indices of the nearest neighbors: [[    1 13528 35297 22534 37542]]
Query results: [{'Unnamed: 0': 2, 'id': 4010796957, 'sequence_number': 7, 'type_id': 20132, 'type_text': 'Step Back Jump Shot', 'text': 'Kristi Toliver makes 18-foot step back jumpshot (Elena Delle Donne assists)', 'away_score': 0, 'home_score': 2, 'period_number': 1, 'period_display_value': '1st Quarter', 'clock_display_value': '9:41', 'scoring_play': True, 'score_value': 2, 'team_id': 16.0, 'athlete_id_1': 856.0, 'athlete_id_2': 2491877.0, 'athlete_id_3': nan, 'wallclock': '2018-09-13T00:04:48Z', 'shooting_play': True, 'coordinate_x_raw': 11, 'coordinate_y_raw': 11.0, 'season': 2018, 'season_type': 3, 'away_team_id': 14, 'away_team_name': 'Seattle', 'away_team_mascot': 'Storm', 'away_team_abbrev': 'SEA', 'away_team_name_alt': 'Seattle', 'home_team_id': 16, 'home_team_name': 'Washington', 'home_team_mascot': 'Mystics', 'home_team_abbrev': 'WSH', 'home_team_name_alt': 'Washington', 'home_team_spread': 2.5, 'game_spread

In [45]:
# Process the result rows to find the specific answer
for index, row in enumerate(result_rows):
    #print(f"Index: {I[index]}")
    print(f"Type: {row['type_text']}")
    print(f"Description: {row['text']}")
    print(f"Away Score: {row['away_score']}")
    print(f"Home Score: {row['home_score']}")
    print(f"Period: {row['period_display_value']}")
    print(f"Clock: {row['clock_display_value']}")
    print("\n")

Type: Step Back Jump Shot
Description: Kristi Toliver makes 18-foot step back jumpshot (Elena Delle Donne assists)
Away Score: 0
Home Score: 2
Period: 1st Quarter
Clock: 9:41


Type: Step Back Jump Shot
Description: Kristi Toliver makes 21-foot step back jumpshot
Away Score: 0
Home Score: 7
Period: 1st Quarter
Clock: 7:26


Type: Step Back Jump Shot
Description: Endy Miyem makes 13-foot step back jumpshot (Cecilia Zandalasini assists)
Away Score: 82
Home Score: 53
Period: 4th Quarter
Clock: 4:40


Type: Step Back Jump Shot
Description: Maya Moore makes 16-foot step back jumpshot (Rebekkah Brunson assists)
Away Score: 4
Home Score: 5
Period: 1st Quarter
Clock: 7:39


Type: Step Back Jump Shot
Description: Candace Parker makes 19-foot step back jumpshot
Away Score: 64
Home Score: 61
Period: 3rd Quarter
Clock: 3:04


